In [1]:
import torch
import os
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_curve

In [2]:
from shutil import copy2

In [3]:
import matplotlib.pyplot as plt

In [4]:
races = ['African','Asian','Caucasian','Indian']
genders = ['Man','Woman']
labels = []
dist = []
for race in races:
    for gender in genders:
        with open(f"./sims/{race}_{gender}_sims.csv",'r') as f:
            lines = f.readlines()
        for line in lines:
            arr = line.strip().split(',')
            labels.append(int(arr[3]))
            dist.append(float(arr[2]))

In [5]:
fpr , tpr ,thresholds = roc_curve(labels,dist)
idx = np.argmax(tpr-fpr)
threshold = thresholds[idx]
print(f"Threshold: {threshold}")

Threshold: 0.2952694594860077


In [6]:
races = ['African','Asian','Caucasian','Indian']
genders = ['Man','Woman']
accuracies = {}
for race in races:
    for gender in genders:
        accuracies[f"{race}_{gender}"] = {'TP':0,'TN':0,'FP':0,'FN':0}
        with open(f"./sims/{race}_{gender}_sims.csv",'r') as f:
            lines = f.readlines()
        dist = []
        labels = []
        for line in lines:
            arr = line.strip().split(',')
            dist.append(float(arr[2]))
            labels.append(int(arr[3]))
            if float(arr[2]) < threshold:
                with open(f"./results/{race}_{gender}_results.csv",'a') as f:
                    f.write(f"{arr[0]},{arr[1]},0,{arr[3]}\n")
                if int(arr[3]) == 1:
                    accuracies[f"{race}_{gender}"]['TN'] += 1
                else:
                    accuracies[f"{race}_{gender}"]['FN'] += 1
            else:
                with open(f"./results/{race}_{gender}_results.csv",'a') as f:
                    f.write(f"{arr[0]},{arr[1]},1,{arr[3]}\n")
                if int(arr[3]) == 1:
                    accuracies[f"{race}_{gender}"]['TP'] += 1
                else:
                    accuracies[f"{race}_{gender}"]['FP'] += 1
        fpr, tpr, thresholds = roc_curve(labels,dist)
        fpr = np.array(fpr)
        tpr = np.array(tpr)
        with open(f"./roc/{race}_{gender}_roc.npy",'wb') as f:
            np.save(f,fpr)
            np.save(f,tpr)

In [7]:
print(accuracies)

{'African_Man': {'TP': 2832, 'TN': 107, 'FP': 66, 'FN': 2874}, 'African_Woman': {'TP': 56, 'TN': 5, 'FP': 1, 'FN': 59}, 'Asian_Man': {'TP': 2022, 'TN': 106, 'FP': 27, 'FN': 2104}, 'Asian_Woman': {'TP': 807, 'TN': 65, 'FP': 8, 'FN': 861}, 'Caucasian_Man': {'TP': 2180, 'TN': 73, 'FP': 3, 'FN': 2249}, 'Caucasian_Woman': {'TP': 716, 'TN': 31, 'FP': 0, 'FN': 748}, 'Indian_Man': {'TP': 2234, 'TN': 81, 'FP': 34, 'FN': 2283}, 'Indian_Woman': {'TP': 664, 'TN': 21, 'FP': 10, 'FN': 672}}


In [8]:
gar = {}
far = {}
scores = {}
for race in races:
    for gender in genders:
        tp = accuracies[f"{race}_{gender}"]['TP']
        tn = accuracies[f"{race}_{gender}"]['TN']
        fp = accuracies[f"{race}_{gender}"]['FP']
        fn = accuracies[f"{race}_{gender}"]['FN']
        gar[f"{race}_{gender}"] = tp/(tn+tp)
        far[f"{race}_{gender}"] = fp/(fn+fp)
        scores[f"{race}_{gender}"] = 100*(fn+tp)/(fn+fp+tn+tp)

In [9]:
gar

{'African_Man': 0.9635930588635591,
 'African_Woman': 0.9180327868852459,
 'Asian_Man': 0.950187969924812,
 'Asian_Woman': 0.9254587155963303,
 'Caucasian_Man': 0.9675987572126055,
 'Caucasian_Woman': 0.9585006693440429,
 'Indian_Man': 0.9650107991360691,
 'Indian_Woman': 0.9693430656934306}

In [10]:
far

{'African_Man': 0.022448979591836733,
 'African_Woman': 0.016666666666666666,
 'Asian_Man': 0.012670107930549039,
 'Asian_Woman': 0.009205983889528193,
 'Caucasian_Man': 0.0013321492007104796,
 'Caucasian_Woman': 0.0,
 'Indian_Man': 0.0146741476046612,
 'Indian_Woman': 0.01466275659824047}

In [11]:
scores

{'African_Man': 97.05732267392413,
 'African_Woman': 95.04132231404958,
 'Asian_Man': 96.87720122094389,
 'Asian_Woman': 95.807007466973,
 'Caucasian_Man': 98.31298557158712,
 'Caucasian_Woman': 97.92642140468227,
 'Indian_Man': 97.51727115716753,
 'Indian_Woman': 97.73226042428676}